# Realizando importações dos pacotes que serão utilizados

In [59]:
import pandas as pd
import numpy as np
from unicodedata import normalize

In [60]:
table_MN = pd.read_csv('data_12_22.CSV', encoding = "ISO-8859-1", sep=";")

In [61]:
df = table_MN[['Ação','Empresa','Preço','Margem EBIT','P/VPA','EV/EBIT','Div.Yield','Volume Financ.(R$)']]

In [62]:
df.head()

,Ação,Empresa,Preço,Margem EBIT,P/VPA,EV/EBIT,Div.Yield,Volume Financ.(R$)
0,ZAMP3,ZAMP S.A.,"4,53","2,11%","0,86","24,04","0,00%",4.560.026
1,YDUQ3,YDUQS PART,"9,61","12,65%","0,98","12,45","1,32%",85.351.838
2,WLMM4,WLM IND COM,"21,9","7,94%","1,32","5,14","4,72%",35.136
3,WLMM3,WLM IND COM,"22,17","7,94%","1,33","5,14","4,24%",376
4,WIZS3,WIZ S.A.,"6,97","41,06%","1,14","3,18","0,00%",10.308.608


# Aplicando transformações nos dados

In [63]:
df['Preço'] = df['Preço'].str.replace('.','')
df['Preço'] = df['Preço'].str.replace(',','.')
df['Preço'] = pd.to_numeric(df['Preço'])

<ipython-input-63-155b712d5614>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Preço'] = df['Preço'].str.replace('.','')


In [64]:
df['P/VPA'] = df['P/VPA'].str.replace('.','')
df['P/VPA'] = df['P/VPA'].str.replace(',','.')
df['P/VPA'] = pd.to_numeric(df['P/VPA'])

<ipython-input-64-a1f0db3b79a8>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['P/VPA'] = df['P/VPA'].str.replace('.','')


In [65]:
df['EV/EBIT'] = df['EV/EBIT'].str.replace('.','')
df['EV/EBIT'] = df['EV/EBIT'].str.replace(',','.')
df['EV/EBIT'] = pd.to_numeric(df['EV/EBIT'])

<ipython-input-65-511aff7950dd>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['EV/EBIT'] = df['EV/EBIT'].str.replace('.','')


In [66]:
df['Volume Financ.(R$)'] = df['Volume Financ.(R$)'].str.replace('.','')
df['Volume Financ.(R$)'] = df['Volume Financ.(R$)'].str.replace(',','.')
df['Volume Financ.(R$)'] = pd.to_numeric(df['Volume Financ.(R$)'])

<ipython-input-66-ad6b51be6698>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Volume Financ.(R$)'] = df['Volume Financ.(R$)'].str.replace('.','')


In [67]:
df['Margem EBIT'] = df['Margem EBIT'].str.replace('%','')
df['Margem EBIT'] = df['Margem EBIT'].str.replace('.','')
df['Margem EBIT'] = df['Margem EBIT'].str.replace(',','.')
df['Margem EBIT'] = pd.to_numeric(df['Margem EBIT'])

<ipython-input-67-a0b0366bde48>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Margem EBIT'] = df['Margem EBIT'].str.replace('.','')


In [68]:
df['Div.Yield'] = df['Div.Yield'].str.replace('%','')
df['Div.Yield'] = df['Div.Yield'].str.replace('.','')
df['Div.Yield'] = df['Div.Yield'].str.replace(',','.')
df['Div.Yield'] = pd.to_numeric(df['Div.Yield'])

<ipython-input-68-49cf0c94ab0a>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Div.Yield'] = df['Div.Yield'].str.replace('.','')


# Removendo empresas pouco negociadas

In [69]:
df = df.drop(df[df['Volume Financ.(R$)'] < 1000000].index)

# Removendo empresas com margem operacional negativo
### Justificativa
- Empresas com o EBIT negativo não dão lucro operacional. Não queremos comprar ações de empresas que dão prejuizo operacional, pois elas são muito arricadas.

In [70]:
# df[df['Margem EBIT'] < 0].head(10)
df = df.drop(df[df['Margem EBIT'] < 0].index)

# Removendo empresas sem margem operacional
### Justificativa
- Geralmente são bancos ou seguradoras.

In [71]:
# df[df['Margem EBIT'].isna()].head(10)
df = df.drop(df[df['Margem EBIT'].isna()].index)

# Removendo empresas duplicadas com menor valor negocivel

In [73]:
df['Ação2'] = df['Ação'].str.replace(r'\d','')
volume_fin_maxes = df.groupby(['Ação2'])['Volume Financ.(R$)'].transform(max)
df = df.loc[df['Volume Financ.(R$)'] == volume_fin_maxes]

<ipython-input-73-ae09750eecd0>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Ação2'] = df['Ação'].str.replace(r'\d','')


# Ordenando os dados pelos menores EV/EBIT

In [77]:
df = df.sort_values(by=['EV/EBIT'], ascending=True)
df = df.reset_index(drop=True)

# Mostrando as ações mais baratas da bolsa

In [78]:
acoes_baratas = df[['Ação','Empresa','EV/EBIT','Div.Yield','Volume Financ.(R$)']]
acoes_baratas.head(50)

,Ação,Empresa,EV/EBIT,Div.Yield,Volume Financ.(R$)
0,INEP3,INEPAR,0.76,0.00,7261710
1,SYNE3,SYN PROP TEC,0.78,218.92,1785235
2,GOAU4,GERDAU MET,0.86,9.46,114159465
3,ENAT3,ENAUTA PART,1.50,14.30,20565606
4,LVTC3,WDC NETWORKS,1.62,3.16,1074577
5,USIM5,USIMINAS,1.70,8.56,148248782
6,PETR4,PETROBRAS,1.92,76.04,2902925613
7,TASA4,TAURUS ARMAS,2.04,12.63,12220604
8,FHER3,FER HERINGER,2.54,0.00,3224854
9,ETER3,ETERNIT,2.58,11.39,9878257
